In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_path = '../input/house-prices-advanced-regression-techniques/train.csv'
test_path = '../input/house-prices-advanced-regression-techniques/test.csv'



train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)


In [3]:
from sklearn.preprocessing import normalize


def process_data(data):

    num_col = data.select_dtypes(include = ['float64', 'int64']).columns
    nnum_col = data.select_dtypes(include =['object']).columns
    

    
    data.loc[:, num_col] = data.select_dtypes(include = ['float64', 'int64'])\
        .fillna(value = data.select_dtypes(include = ['float64', 'int64']).median())
    data.loc[:, nnum_col] = data.select_dtypes(include =['object']).fillna(value = 'N')

    for label, content in data.select_dtypes(include = ['float64', 'int64']).items():
        if label == 'Id' or label =='SalePrice':
            continue

        meanV = content.mean()
        stdV = content.std()
        data[label] = (content - meanV)/stdV

    
    column_names = data.select_dtypes(include =['object']).columns
    data = pd.get_dummies(data, columns = column_names)

    return data


train_data = process_data(train_data)
test_data = process_data(test_data)

missing_in_train = []
missing_in_test  = []

for col in test_data.columns:
    if col not in train_data.columns:
        missing_in_train.append(col)
        
for col in train_data.columns:
    if col not in test_data.columns and col != 'SalePrice':
        missing_in_test.append(col)      
        

# print(missing_in_train)
# print(missing_in_test)

train_data[missing_in_train] = 0
test_data[missing_in_test] = 0


In [4]:
import math


corr_epsilon= 0.0
t = train_data.corr().abs()['SalePrice'].sort_values(ascending=False)
features1 =t[t>corr_epsilon].index
features2 = t[t.isnull()].index
features = features1.union(features2)
# features = features.insert(1,'Id')


train_data = train_data[features]
test_data = test_data[features[features!='SalePrice']]

# train_data.head()
train_data.Id

0          1
1          2
2          3
3          4
4          5
        ... 
1455    1456
1456    1457
1457    1458
1458    1459
1459    1460
Name: Id, Length: 1460, dtype: int64

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler


# train, xtest, ytrain, ytest = train_test_split(X, y.iloc[:,1], test_size=1/3,
#   random_state=85, stratify=y.iloc[:,1])

X_train = train_data.loc[:, train_data.columns!='SalePrice']
y_train = train_data.SalePrice

X_test = test_data



In [6]:
from sklearn import svm
from sklearn.pipeline import make_pipeline


# svm_model= make_pipeline(StandardScaler(), svm.SVR())
# cv = cross_val_score(svm_model,X_train,y_train,cv=5, scoring ='neg_root_mean_squared_error')
# print(cv)
# print(cv.mean())

# svm_model.fit(X_train, y_train)
# svm_model.predict(X_test)

In [7]:
from sklearn.linear_model import SGDRegressor



# sgdr = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1e3, tol=1e-3))
# cv = cross_val_score(sgdr,X_train,y_train,cv=5, scoring ='neg_root_mean_squared_error')
# print(cv)
# print(cv.mean()) 

In [8]:
from sklearn.ensemble import GradientBoostingRegressor

# GBR = make_pipeline(StandardScaler(),\
#                     GradientBoostingRegressor(n_estimators = 100, learning_rate =0.1, max_depth =2 , random_state =0))

# cv = cross_val_score(GBR, X_train, y_train, cv=5, scoring = 'neg_root_mean_squared_error')
# print(cv)
# print(cv.mean()) 


In [9]:
def performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_) )
    print('Best Parameters: ' + str(classifier.best_params_))
    print()

Initial parameters models, scoring = neg root mean squared error:
* SGDRegressor  -16775592
* Support Vector Regression -81141
* GradientBoostingRegressor -27360

In [10]:
from sklearn.model_selection import GridSearchCV

# svr = svm.SVR()

# param_grid= {'kernel': ['linear', 'poly', 'sigmoid', 'rbf'],
#              'tol': [1e-3],
#              'epsilon': [0.1,0.2,0.3, 0.05, 0.4,0.5,0.6,0.7,1.0,5.0,10.0,20.0 ],
#              'gamma': ['scale']
#             }

# search_svr = GridSearchCV(svr, param_grid=param_grid, cv= 5, verbose = True, n_jobs=-1, scoring = 'neg_root_mean_squared_error')
# best_perf = search_svr.fit(X_train, y_train)
# performance(best_perf, 'Support Vector Regression')

Fitting 5 folds for each of 48 candidates, totalling 240 fits<br>
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.<br>
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.7s<br>
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   51.6s<br>
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.1min finished<br>
Support Vector Regression<br>
Best Score: -74890.33216153785<br>
Best Parameters: {'epsilon': 20.0, 'gamma': 'scale', 'kernel': 'linear', 'tol': 0.001}<br>

In [11]:
# GBR = GradientBoostingRegressor()
# param_grid= {'learning_rate': [0.05],
#              'n_estimators' : [1000 ],
#              'subsample': [1.0],
#              'max_features': ['sqrt'],
#              'max_depth': [1,2,3,4,5,6,7,8,9,10]
#             }

# search_GBR = GridSearchCV(GBR, param_grid=param_grid, cv=5, verbose=True, n_jobs=-1, scoring ='neg_root_mean_squared_error' )
# best_perf = search_GBR.fit(X_train, y_train)
# performance(best_perf, 'Gradient Boosting Regressor')

Gradient Boosting Regressor<br>
Best Score: -25194.760742077844<br>
Best Parameters: {'learning_rate': 0.05, 'max_features': 'sqrt', 'n_estimators': 1000,subsample': 1.0, 'max_depth': 3}<br>

In [12]:

GBR = make_pipeline(StandardScaler(),\
                    GradientBoostingRegressor(n_estimators = 1000,subsample = 1.0,\
                                              learning_rate =0.05, max_depth =3 , \
                                              random_state =0, max_features= 'sqrt'))

# 'neg_root_mean_squared_error'

cv = cross_val_score(GBR, X_train, y_train, cv=5, scoring = 'neg_root_mean_squared_error')
print(cv)
print(cv.mean()) 


GBR.fit(X_train,y_train)

predictions = GBR.predict(X_test)

output = pd.DataFrame({'Id': X_test.Id, 'SalePrice': predictions})
output.to_csv('submission.csv', index=False)

[-21021.37410632 -32174.30186133 -26072.44434707 -20246.12021164
 -29849.86585703]
-25872.82127668037
